# Machine Learning Algorithms for Network Intrusion Detection
## Comparative Evaluation with Hyperparameter Optimization

**Course:** SIM600 - Cloud Security  
**Dataset:** UNSW-NB15  
**Methodology:** Experimental Design with proper train/test separation

---

## Methodology Overview

This notebook implements a **rigorous experimental research design** following IEEE standards:

1. **Data Loading**: Load pre-split train/test sets (maintain separation)
2. **Preprocessing**: Fit transformations on train set, apply to both
3. **Baseline Evaluation**: Evaluate default algorithms on train set with 5-fold CV
4. **Hyperparameter Tuning**: Grid search on train set only using CV
5. **Final Evaluation**: Test optimized models on independent test set
6. **Results Export**: Generate publication-ready metrics and visualizations

**Key Principle**: Test data is NEVER seen during training or hyperparameter tuning to ensure unbiased performance estimates.

## 1. Environment Setup

In [1]:
# Install Required Packages
#%pip install -q seaborn psutil memory_profiler scikit-learn matplotlib pandas numpy

# Memory optimization
import gc
import os

gc.set_threshold(700, 10, 10)

print("✅ Environment configured for optimal performance")

✅ Environment configured for optimal performance


In [2]:
# Import Libraries
import time
import pickle
import warnings
warnings.filterwarnings('ignore')
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
from memory_profiler import memory_usage

# Sklearn imports
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!\n")
print("="*70)
print("SYSTEM INFO")
print("="*70)
print(f"CPU Cores: {os.cpu_count()}")
print(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")
print("="*70)

✅ All libraries imported successfully!

SYSTEM INFO
CPU Cores: 12
Total RAM: 15.93 GB
Available RAM: 6.37 GB


## 2. Data Loading

**CRITICAL**: We load train and test sets separately and keep them separated throughout the entire pipeline.

**Note**: The UNSW-NB15 files appear to have swapped labels:
- UNSW_NB15_training-set.csv contains 82,332 samples (smaller set)
- UNSW_NB15_testing-set.csv contains 175,341 samples (larger set)

We'll use the larger file for training and smaller for testing, which is the standard practice.

In [3]:
print("Loading UNSW-NB15 datasets...\n")

# Load datasets - NOTE: Files appear to have swapped naming
# Using larger set for training, smaller for testing
train_raw = pd.read_csv('UNSW_NB15_testing-set.csv')  # 175,341 samples - LARGER
test_raw = pd.read_csv('UNSW_NB15_training-set.csv')   # 82,332 samples - SMALLER

print("="*70)
print("DATASET LOADING COMPLETE")
print("="*70)
print(f"Training set: {train_raw.shape[0]:,} samples, {train_raw.shape[1]} features")
print(f"Test set:     {test_raw.shape[0]:,} samples, {test_raw.shape[1]} features")
print(f"Total:        {train_raw.shape[0] + test_raw.shape[0]:,} samples")

# Analyze class distribution
print("\n" + "="*70)
print("CLASS DISTRIBUTION")
print("="*70)

train_attacks = train_raw['label'].sum()
train_normal = len(train_raw) - train_attacks
test_attacks = test_raw['label'].sum()
test_normal = len(test_raw) - test_attacks

print(f"\nTraining Set:")
print(f"  Attack samples:  {train_attacks:>6,} ({train_attacks/len(train_raw)*100:>5.2f}%)")
print(f"  Normal samples:  {train_normal:>6,} ({train_normal/len(train_raw)*100:>5.2f}%)")

print(f"\nTest Set:")
print(f"  Attack samples:  {test_attacks:>6,} ({test_attacks/len(test_raw)*100:>5.2f}%)")
print(f"  Normal samples:  {test_normal:>6,} ({test_normal/len(test_raw)*100:>5.2f}%)")
print("="*70)

# Store original copies
train_original = train_raw.copy()
test_original = test_raw.copy()

Loading UNSW-NB15 datasets...

DATASET LOADING COMPLETE
Training set: 175,341 samples, 45 features
Test set:     82,332 samples, 45 features
Total:        257,673 samples

CLASS DISTRIBUTION

Training Set:
  Attack samples:  119,341 (68.06%)
  Normal samples:  56,000 (31.94%)

Test Set:
  Attack samples:  45,332 (55.06%)
  Normal samples:  37,000 (44.94%)


## 3. Data Preprocessing

**Methodology**: 
1. Fit preprocessing on training set only
2. Apply same transformations to both train and test
3. This prevents data leakage from test set

In [4]:
def preprocess_data(train_df, test_df):
    """
    Preprocess train and test sets consistently with class balancing.
    
    IMPORTANT: All transformations are fit on train set only,
    then applied to both train and test sets.
    Classes are balanced using undersampling for both train and test.
    
    Returns:
        X_train, y_train, X_test, y_test
    """
    print("\n" + "="*70)
    print("PREPROCESSING PIPELINE WITH CLASS BALANCING (UNDERSAMPLING)")
    print("="*70)
    
    # Step 1: Remove unnecessary columns
    print("\n1. Removing ID and attack_cat columns...")
    cols_to_drop = ['id', 'attack_cat']
    train_clean = train_df.drop(cols_to_drop, axis=1)
    test_clean = test_df.drop(cols_to_drop, axis=1)
    
    # Step 2: Identify numeric columns BEFORE encoding
    print("2. Identifying numeric columns for normalization...")
    numeric_cols_before_encoding = train_clean.select_dtypes(include=[np.number]).columns.tolist()
    if 'label' in numeric_cols_before_encoding:
        numeric_cols_before_encoding.remove('label')
    print(f"   Found {len(numeric_cols_before_encoding)} numeric columns to normalize")
    
    # Step 3: Handle categorical variables
    print("3. Processing categorical features...")
    
    # Identify categorical columns
    cat_cols = train_clean.select_dtypes('object').columns.tolist()
    print(f"   Found {len(cat_cols)} categorical columns: {cat_cols}")
    
    # Replace '-' with 'None' in categorical columns
    for col in cat_cols:
        train_clean[col] = train_clean[col].replace('-', 'None')
        test_clean[col] = test_clean[col].replace('-', 'None')
    
    # One-hot encoding (fit on train, apply to both)
    print("4. Performing one-hot encoding...")
    train_encoded = pd.get_dummies(train_clean, columns=cat_cols, drop_first=False)
    test_encoded = pd.get_dummies(test_clean, columns=cat_cols, drop_first=False)
    
    # Align columns (test might have different categories)
    train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)
    
    print(f"   Shape after encoding: {train_encoded.shape}")
    
    # Step 4: Separate features and labels
    print("5. Separating features and labels...")
    X_train = train_encoded.drop('label', axis=1)
    y_train = train_encoded['label'].astype(int)
    X_test = test_encoded.drop('label', axis=1)
    y_test = test_encoded['label'].astype(int)
    
    # Step 5: Normalize ONLY the original numeric features
    print("6. Normalizing numeric features (fit on train only)...")
    print("   Note: One-hot encoded features are already 0/1 and don't need normalization")
    
    # Identify which columns correspond to original numeric columns
    numeric_cols_to_normalize = [col for col in X_train.columns if col in numeric_cols_before_encoding]
    print(f"   Normalizing {len(numeric_cols_to_normalize)} numeric columns")
    
    from sklearn.preprocessing import StandardScaler

    if len(numeric_cols_to_normalize) > 0:
            
        scaler = StandardScaler()
        
        # Fit on train, transform both (StandardScaler handles zero std internally)
        X_train[numeric_cols_to_normalize] = scaler.fit_transform(X_train[numeric_cols_to_normalize])
        X_test[numeric_cols_to_normalize] = scaler.transform(X_test[numeric_cols_to_normalize])
        
        # Safety check: Replace infinities and NaNs with 0 (unlikely but defensive)
        X_train[numeric_cols_to_normalize] = X_train[numeric_cols_to_normalize].replace([np.inf, -np.inf], 0).fillna(0)
        X_test[numeric_cols_to_normalize] = X_test[numeric_cols_to_normalize].replace([np.inf, -np.inf], 0).fillna(0)
        
        # Convert all columns to float for consistency
        X_train_normalized = X_train.astype(float)
        X_test_normalized = X_test.astype(float)
        
        # Step 6: Balance classes by downsampling majority class
        print("\n7. Balancing classes by downsampling majority class...")
        
        # Check class distribution before balancing
        print("\n   BEFORE BALANCING:")
        train_class_0 = (y_train == 0).sum()
        train_class_1 = (y_train == 1).sum()
        test_class_0 = (y_test == 0).sum()
        test_class_1 = (y_test == 1).sum()
        print(f"   Training set - Class 0: {train_class_0}, Class 1: {train_class_1}")
        print(f"   Test set     - Class 0: {test_class_0}, Class 1: {test_class_1}")
    
    # Balance training set
    if train_class_0 > train_class_1:
        # Class 0 is majority, downsample it to match class 1
        minority_count = train_class_1
        majority_indices = y_train[y_train == 0].index
        minority_indices = y_train[y_train == 1].index
        
        # Randomly sample from majority class
        np.random.seed(42)
        downsampled_majority = np.random.choice(majority_indices, size=minority_count, replace=False)
        
        # Combine indices
        balanced_indices = np.concatenate([downsampled_majority, minority_indices])
        
        X_train_balanced = X_train_normalized.iloc[balanced_indices]
        y_train_balanced = y_train.iloc[balanced_indices]
    else:
        # Class 1 is majority, downsample it to match class 0
        minority_count = train_class_0
        majority_indices = y_train[y_train == 1].index
        minority_indices = y_train[y_train == 0].index
        
        # Randomly sample from majority class
        np.random.seed(42)
        downsampled_majority = np.random.choice(majority_indices, size=minority_count, replace=False)
        
        # Combine indices
        balanced_indices = np.concatenate([downsampled_majority, minority_indices])
        
        X_train_balanced = X_train_normalized.iloc[balanced_indices]
        y_train_balanced = y_train.iloc[balanced_indices]
    
    # Balance test set
    if test_class_0 > test_class_1:
        # Class 0 is majority, downsample it to match class 1
        minority_count = test_class_1
        majority_indices = y_test[y_test == 0].index
        minority_indices = y_test[y_test == 1].index
        
        # Randomly sample from majority class
        np.random.seed(42)
        downsampled_majority = np.random.choice(majority_indices, size=minority_count, replace=False)
        
        # Combine indices
        balanced_indices = np.concatenate([downsampled_majority, minority_indices])
        
        X_test_balanced = X_test_normalized.iloc[balanced_indices]
        y_test_balanced = y_test.iloc[balanced_indices]
    else:
        # Class 1 is majority, downsample it to match class 0
        minority_count = test_class_0
        majority_indices = y_test[y_test == 1].index
        minority_indices = y_test[y_test == 0].index
        
        # Randomly sample from majority class
        np.random.seed(42)
        downsampled_majority = np.random.choice(majority_indices, size=minority_count, replace=False)
        
        # Combine indices
        balanced_indices = np.concatenate([downsampled_majority, minority_indices])
        
        X_test_balanced = X_test_normalized.iloc[balanced_indices]
        y_test_balanced = y_test.iloc[balanced_indices]
    
    # Check class distribution after balancing
    print("\n   AFTER BALANCING:")
    print(f"   Training set - Class 0: {(y_train_balanced == 0).sum()}, Class 1: {(y_train_balanced == 1).sum()}")
    print(f"   Test set     - Class 0: {(y_test_balanced == 0).sum()}, Class 1: {(y_test_balanced == 1).sum()}")
    
    print("\n" + "="*70)
    print("PREPROCESSING COMPLETE")
    print("="*70)
    print(f"Training features: {X_train_balanced.shape}")
    print(f"Training labels:   {y_train_balanced.shape}")
    print(f"Test features:     {X_test_balanced.shape}")
    print(f"Test labels:       {y_test_balanced.shape}")
    print("="*70 + "\n")
    
    return X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced

# Preprocess data
X_train, y_train, X_test, y_test = preprocess_data(train_original, test_original)

# Force garbage collection
del train_raw, test_raw, train_original, test_original
gc.collect()

print("✅ Data preprocessing complete. Train and test sets ready with balanced classes.")


PREPROCESSING PIPELINE WITH CLASS BALANCING (UNDERSAMPLING)

1. Removing ID and attack_cat columns...
2. Identifying numeric columns for normalization...
   Found 39 numeric columns to normalize
3. Processing categorical features...
   Found 3 categorical columns: ['proto', 'service', 'state']
4. Performing one-hot encoding...
   Shape after encoding: (175341, 195)
5. Separating features and labels...
6. Normalizing numeric features (fit on train only)...
   Note: One-hot encoded features are already 0/1 and don't need normalization
   Normalizing 39 numeric columns

7. Balancing classes by downsampling majority class...

   BEFORE BALANCING:
   Training set - Class 0: 56000, Class 1: 119341
   Test set     - Class 0: 37000, Class 1: 45332

   AFTER BALANCING:
   Training set - Class 0: 56000, Class 1: 56000
   Test set     - Class 0: 37000, Class 1: 37000

PREPROCESSING COMPLETE
Training features: (112000, 194)
Training labels:   (112000,)
Test features:     (74000, 194)
Test labels:

## 4. Model Evaluation Functions

We define comprehensive evaluation functions that:
1. Train models on training set
2. Evaluate on both train (with CV) and test sets
3. Calculate cloud deployment metrics

In [5]:
def get_model_size_mb(model):
    """Calculate model size in MB"""
    try:
        model_bytes = pickle.dumps(model)
        return len(model_bytes) / (1024 * 1024)
    except:
        return 0.0

def evaluate_model_comprehensive(model, X_train, y_train, X_test, y_test, 
                                  model_name="Model", cv=2):
    """
    Comprehensive model evaluation with proper train/test separation.
    
    Args:
        model: Sklearn model instance
        X_train, y_train: Training data
        X_test, y_test: Test data (held-out, unseen)
        model_name: Name for reporting
        cv: Number of cross-validation folds for train evaluation
    
    Returns:
        results dict with train and test metrics
    """
    
    print(f"\n{'='*70}")
    print(f"EVALUATING: {model_name}")
    print(f"{'='*70}")
    
    results = {'model_name': model_name}
    
    # Get baseline memory
    process = psutil.Process(os.getpid())
    baseline_memory = process.memory_info().rss / (1024 * 1024)
    
    # ========================================
    # TRAINING PHASE
    # ========================================
    print("\n[1/5] Training model on training set...")
    train_start = time.time()
    
    def train_model():
        return model.fit(X_train, y_train)
    
    mem_usage = memory_usage((train_model,), interval=0.1, timeout=None)
    trained_model = model.fit(X_train, y_train)
    
    train_time = time.time() - train_start
    peak_memory = max(mem_usage)
    memory_used = peak_memory - baseline_memory
    
    results['training_time_sec'] = round(train_time, 3)
    results['memory_mb'] = round(memory_used, 2)
    
    # ========================================
    # CROSS-VALIDATION ON TRAINING SET
    # ========================================
    print(f"[2/5] Cross-validation on training set (cv={cv})...")
    cv_start = time.time()
    
    y_train_pred_cv = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=-1)
    
    cv_time = time.time() - cv_start
    results['cv_time_sec'] = round(cv_time, 3)
    
    # Calculate train metrics (cross-validated)
    results['train_accuracy'] = round(metrics.accuracy_score(y_train, y_train_pred_cv) * 100, 2)
    results['train_precision'] = round(metrics.precision_score(y_train, y_train_pred_cv, zero_division=0) * 100, 2)
    results['train_recall'] = round(metrics.recall_score(y_train, y_train_pred_cv, zero_division=0) * 100, 2)
    results['train_f1'] = round(metrics.f1_score(y_train, y_train_pred_cv, zero_division=0) * 100, 2)
    
    # Train confusion matrix
    cm_train = metrics.confusion_matrix(y_train, y_train_pred_cv)
    tn_train, fp_train, fn_train, tp_train = cm_train.ravel()
    results['train_tn'] = int(tn_train)
    results['train_fp'] = int(fp_train)
    results['train_fn'] = int(fn_train)
    results['train_tp'] = int(tp_train)
    results['train_fpr'] = round((fp_train / (fp_train + tn_train) * 100) if (fp_train + tn_train) > 0 else 0, 2)
    
    # ========================================
    # TEST SET EVALUATION (CRITICAL)
    # ========================================
    print("[3/5] Evaluating on independent test set...")
    test_start = time.time()
    
    y_test_pred = trained_model.predict(X_test)
    
    test_time = time.time() - test_start
    results['test_time_sec'] = round(test_time, 3)
    results['avg_latency_ms'] = round((test_time / len(X_test)) * 1000, 4)
    
    # Calculate test metrics
    results['test_accuracy'] = round(metrics.accuracy_score(y_test, y_test_pred) * 100, 2)
    results['test_precision'] = round(metrics.precision_score(y_test, y_test_pred, zero_division=0) * 100, 2)
    results['test_recall'] = round(metrics.recall_score(y_test, y_test_pred, zero_division=0) * 100, 2)
    results['test_f1'] = round(metrics.f1_score(y_test, y_test_pred, zero_division=0) * 100, 2)
    
    # Test confusion matrix
    cm_test = metrics.confusion_matrix(y_test, y_test_pred)
    tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
    results['test_tn'] = int(tn_test)
    results['test_fp'] = int(fp_test)
    results['test_fn'] = int(fn_test)
    results['test_tp'] = int(tp_test)
    results['test_fpr'] = round((fp_test / (fp_test + tn_test) * 100) if (fp_test + tn_test) > 0 else 0, 2)
    
    # Store confusion matrices
    results['cm_train'] = cm_train
    results['cm_test'] = cm_test
    
    # ========================================
    # CLOUD DEPLOYMENT METRICS
    # ========================================
    print("[4/5] Calculating deployment metrics...")
    
    model_size = get_model_size_mb(trained_model)
    results['model_size_mb'] = round(model_size, 2)
    
    throughput = len(X_test) / test_time if test_time > 0 else 0
    results['throughput_samples_per_sec'] = round(throughput, 2)
    
    # Cost-effectiveness: F1-score per second of training time
    cost_effectiveness = results['test_f1'] / train_time if train_time > 0 else 0
    results['cost_effectiveness'] = round(cost_effectiveness, 2)
    
    # Performance per MB: F1-score per MB of memory
    perf_per_mb = results['test_f1'] / memory_used if memory_used > 0 else 0
    results['performance_per_mb'] = round(perf_per_mb, 2)
    
    # Scalability score
    avg_latency = results['avg_latency_ms']
    scalability = (throughput / 1000) / avg_latency if avg_latency > 0 else 0
    results['scalability_score'] = round(scalability, 4)
    
    # ========================================
    # RESULTS SUMMARY
    # ========================================
    print("[5/5] Results compiled.\n")
    
    print(f"{'TRAINING SET PERFORMANCE (Cross-Validated)':^70}")
    print(f"{'-'*70}")
    print(f"  Accuracy:        {results['train_accuracy']:>6.2f}%")
    print(f"  Precision:       {results['train_precision']:>6.2f}%")
    print(f"  Recall:          {results['train_recall']:>6.2f}%")
    print(f"  F1-Score:        {results['train_f1']:>6.2f}%")
    print(f"  False Pos. Rate: {results['train_fpr']:>6.2f}%")
    
    print(f"\n{'TEST SET PERFORMANCE (Independent Evaluation)':^70}")
    print(f"{'-'*70}")
    print(f"  Accuracy:        {results['test_accuracy']:>6.2f}%")
    print(f"  Precision:       {results['test_precision']:>6.2f}%")
    print(f"  Recall:          {results['test_recall']:>6.2f}%")
    print(f"  F1-Score:        {results['test_f1']:>6.2f}%")
    print(f"  False Pos. Rate: {results['test_fpr']:>6.2f}%")
    
    print(f"\n{'RESOURCE & DEPLOYMENT METRICS':^70}")
    print(f"{'-'*70}")
    print(f"  Training Time:   {results['training_time_sec']:>8.3f} sec")
    print(f"  Test Time:       {results['test_time_sec']:>8.3f} sec")
    print(f"  Avg Latency:     {results['avg_latency_ms']:>8.4f} ms/sample")
    print(f"  Memory Used:     {results['memory_mb']:>8.2f} MB")
    print(f"  Model Size:      {results['model_size_mb']:>8.2f} MB")
    print(f"  Throughput:      {results['throughput_samples_per_sec']:>8.2f} samples/sec")
    print(f"{'='*70}\n")
    
    return results, trained_model

print("✅ Evaluation functions defined")

✅ Evaluation functions defined


## 5. Baseline Model Evaluation

Evaluate three algorithms with default hyperparameters to establish baseline performance.

In [6]:
print("\n" + "="*70)
print("BASELINE EVALUATIONS (Default Parameters)")
print("="*70)
print("\nEvaluating models with 5-fold cross-validation on training set,")
print("then testing on independent test set.\n")

baseline_results = {}
baseline_models = {}


BASELINE EVALUATIONS (Default Parameters)

Evaluating models with 5-fold cross-validation on training set,
then testing on independent test set.



In [7]:
# 1. Logistic Regression (Baseline)
import gc
lr_baseline = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
results_lr_base, model_lr_base = evaluate_model_comprehensive(
    lr_baseline, X_train, y_train, X_test, y_test,
    model_name="Logistic Regression (Baseline)", cv=5
)
baseline_results['Logistic Regression'] = results_lr_base
baseline_models['Logistic Regression'] = model_lr_base

gc.collect()  # Clean memory


EVALUATING: Logistic Regression (Baseline)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=2)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         85.44%
  Precision:        80.32%
  Recall:           93.89%
  F1-Score:         86.57%
  False Pos. Rate:  23.01%

            TEST SET PERFORMANCE (Independent Evaluation)             
----------------------------------------------------------------------
  Accuracy:         82.41%
  Precision:        76.66%
  Recall:           93.20%
  F1-Score:         84.12%
  False Pos. Rate:  28.38%

                    RESOURCE & DEPLOYMENT METRICS                     
----------------------------------------------------------------------
  Training Time:      8.592 sec
  Test Time:          0.045

52

In [8]:
# 2. K-Nearest Neighbors (Baseline)
knn_baseline = KNeighborsClassifier(n_jobs=-1)
results_knn_base, model_knn_base = evaluate_model_comprehensive(
    knn_baseline, X_train, y_train, X_test, y_test,
    model_name="K-Nearest Neighbors (Baseline)", cv=5
)
baseline_results['KNN'] = results_knn_base
baseline_models['KNN'] = model_knn_base

gc.collect()


EVALUATING: K-Nearest Neighbors (Baseline)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=2)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         85.40%
  Precision:        80.60%
  Recall:           93.24%
  F1-Score:         86.46%
  False Pos. Rate:  22.44%

            TEST SET PERFORMANCE (Independent Evaluation)             
----------------------------------------------------------------------
  Accuracy:         86.04%
  Precision:        81.84%
  Recall:           92.65%
  F1-Score:         86.91%
  False Pos. Rate:  20.56%

                    RESOURCE & DEPLOYMENT METRICS                     
----------------------------------------------------------------------
  Training Time:      1.526 sec
  Test Time:         31.205

48

In [9]:
# 3. Linear SVC (Baseline)
svc_baseline = LinearSVC(max_iter=1000, random_state=42)
results_svc_base, model_svc_base = evaluate_model_comprehensive(
    svc_baseline, X_train, y_train, X_test, y_test,
    model_name="Linear SVC (Baseline)", cv=5
)
baseline_results['Linear SVC'] = results_svc_base
baseline_models['Linear SVC'] = model_svc_base

gc.collect()


EVALUATING: Linear SVC (Baseline)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=5)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         88.52%
  Precision:        83.91%
  Recall:           95.33%
  F1-Score:         89.26%
  False Pos. Rate:  18.28%

            TEST SET PERFORMANCE (Independent Evaluation)             
----------------------------------------------------------------------
  Accuracy:         81.73%
  Precision:        75.74%
  Recall:           93.37%
  F1-Score:         83.64%
  False Pos. Rate:  29.90%

                    RESOURCE & DEPLOYMENT METRICS                     
----------------------------------------------------------------------
  Training Time:      9.694 sec
  Test Time:          0.040 sec
  Av

0

## 6. Hyperparameter Optimization

**Grid Search on Training Set Only** using 5-fold cross-validation.

The test set is NOT used during hyperparameter tuning.

In [10]:
def grid_search_optimize(model, param_grid, X_train, y_train, 
                         model_name="Model", cv=5, n_jobs=-1):
    """
    Perform grid search on TRAINING set only.
    
    CRITICAL: Test set is never used here.
    """
    print(f"\n{'='*70}")
    print(f"GRID SEARCH: {model_name}")
    print(f"{'='*70}")
    print(f"Parameter grid: {param_grid}")
    print(f"Cross-validation folds: {cv}")
    print(f"Scoring metric: F1-Score")
    print(f"\nSearching for optimal hyperparameters...")
    
    start_time = time.time()
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        scoring='f1',
        n_jobs=n_jobs,
        verbose=1,
        return_train_score=True
    )
    
    # Fit on TRAINING set only
    grid_search.fit(X_train, y_train)
    
    search_time = time.time() - start_time
    
    print(f"\n{'='*70}")
    print(f"GRID SEARCH COMPLETE - {model_name}")
    print(f"{'='*70}")
    print(f"Search time: {search_time:.2f} seconds")
    print(f"\nBest Parameters:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")
    print(f"\nBest CV F1-Score (Train): {grid_search.best_score_*100:.2f}%")
    print(f"{'='*70}\n")
    
    gc.collect()
    
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_

print("✅ Grid search function defined")

✅ Grid search function defined


### 6.1 Optimize Logistic Regression

In [11]:
print("\n" + "="*70)
print("HYPERPARAMETER TUNING: LOGISTIC REGRESSION")
print("="*70)
lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear', 'SAGA', 'SAG'],
    'max_iter': [1000]
}
# Grid search on TRAINING set only
lr_tuned, lr_best_params, lr_best_cv_score = grid_search_optimize(
    model=LogisticRegression(random_state=42, n_jobs=-1),
    param_grid=lr_param_grid,
    X_train=X_train,
    y_train=y_train,
    model_name="Logistic Regression",
    cv=5
)
# Evaluate on BOTH train and test
results_lr_tuned, model_lr_tuned = evaluate_model_comprehensive(
    lr_tuned, X_train, y_train, X_test, y_test,
    model_name="Logistic Regression (Tuned)", cv=5
)
model_filename = '123Logistic.pkl'
print("\n" + "="*70)
print("SAVING MODEL")
print("="*70)
with open(model_filename, 'wb') as file:
    pickle.dump(model_lr_tuned, file)  # ✓ FIXED: Changed from model_knn_tuned
# Get file size
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)
print(f"✓ Model saved: {model_filename}")
print(f"File size: {file_size_mb:.4f} MB")
print(f"Best parameters: {lr_best_params}")  # ✓ FIXED: Changed from knn_best_params
print(f"Best CV score: {lr_best_cv_score:.4f}")  # ✓ FIXED: Changed from knn_best_cv_score
# Verify the saved model
print("\n--- VERIFYING SAVED MODEL ---")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
from sklearn.metrics import accuracy_score
y_pred_loaded = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"Loaded model test accuracy: {loaded_accuracy:.4f}")
print("✓ Model successfully saved and verified!")
print("\n" + "="*70)
print("PROCESS COMPLETE")
print("="*70)


HYPERPARAMETER TUNING: LOGISTIC REGRESSION

GRID SEARCH: Logistic Regression
Parameter grid: {'C': [100], 'penalty': ['l2'], 'solver': ['liblinear'], 'max_iter': [1000]}
Cross-validation folds: 5
Scoring metric: F1-Score

Searching for optimal hyperparameters...
Fitting 5 folds for each of 1 candidates, totalling 5 fits

GRID SEARCH COMPLETE - Logistic Regression
Search time: 41.13 seconds

Best Parameters:
  C: 100
  max_iter: 1000
  penalty: l2
  solver: liblinear

Best CV F1-Score (Train): 90.65%


EVALUATING: Logistic Regression (Tuned)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=5)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         89.09%
  Precision:        84.91%
  Recall:           95.07%
  F1-Score:         89.70%
  F

### 6.2 Optimize K-Nearest Neighbors

In [12]:
import pickle
import os

print("\n" + "="*70)
print("HYPERPARAMETER TUNING: K-NEAREST NEIGHBORS")
print("="*70)

knn_param_grid = {
    'n_neighbors': [3,5,7,9,11],
    'weights': ['distance', 'uniform'],
    'metric': ['manhattan', 'minkowski', 'euclidean'],
}

# Grid search on TRAINING set only
knn_tuned, knn_best_params, knn_best_cv_score = grid_search_optimize(
    model=KNeighborsClassifier(n_jobs=-1),
    param_grid=knn_param_grid,
    X_train=X_train,
    y_train=y_train,
    model_name="K-Nearest Neighbors",
    cv=5
)

# Evaluate on BOTH train and test
results_knn_tuned, model_knn_tuned = evaluate_model_comprehensive(
    knn_tuned, X_train, y_train, X_test, y_test,
    model_name="K-Nearest Neighbors (Tuned)", cv=5
)

# Save the trained model
model_filename = '123knn_tuned_model.pkl'
print("\n" + "="*70)
print("SAVING MODEL")
print("="*70)

with open(model_filename, 'wb') as file:
    pickle.dump(model_knn_tuned, file)

# Get file size
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)

print(f"✓ Model saved: {model_filename}")
print(f"File size: {file_size_mb:.4f} MB")
print(f"Best parameters: {knn_best_params}")
print(f"Best CV score: {knn_best_cv_score:.4f}")

# Verify the saved model
print("\n--- VERIFYING SAVED MODEL ---")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

from sklearn.metrics import accuracy_score
y_pred_loaded = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"Loaded model test accuracy: {loaded_accuracy:.4f}")
print("✓ Model successfully saved and verified!")

print("\n" + "="*70)
print("PROCESS COMPLETE")
print("="*70)


HYPERPARAMETER TUNING: K-NEAREST NEIGHBORS

GRID SEARCH: K-Nearest Neighbors
Parameter grid: {'n_neighbors': [11], 'weights': ['distance'], 'metric': ['manhattan'], 'algorithm': ['auto']}
Cross-validation folds: 5
Scoring metric: F1-Score

Searching for optimal hyperparameters...
Fitting 5 folds for each of 1 candidates, totalling 5 fits

GRID SEARCH COMPLETE - K-Nearest Neighbors
Search time: 690.70 seconds

Best Parameters:
  algorithm: auto
  metric: manhattan
  n_neighbors: 11
  weights: distance

Best CV F1-Score (Train): 90.46%


EVALUATING: K-Nearest Neighbors (Tuned)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=2)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         85.69%
  Precision:        80.75%
  Recall:           93

### 6.3 Optimize Linear SVC

In [13]:
print("\n" + "="*70)
print("HYPERPARAMETER TUNING: LINEAR SVC")
print("="*70)
svc_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10 100],
    'penalty': ['l2', 'l1'],
    'loss': ['hinge', 'squared_hinge'],
    'max_iter': [1000]
}
# Grid search on TRAINING set only
svc_tuned, svc_best_params, svc_best_cv_score = grid_search_optimize(
    model=LinearSVC(random_state=42),
    param_grid=svc_param_grid,
    X_train=X_train,
    y_train=y_train,
    model_name="Linear SVC",
    cv=5
)
# Evaluate on BOTH train and test
results_svc_tuned, model_svc_tuned = evaluate_model_comprehensive(
    svc_tuned, X_train, y_train, X_test, y_test,
    model_name="Linear SVC (Tuned)", cv=5
)
model_filename = '123SVC.pkl'
print("\n" + "="*70)
print("SAVING MODEL")
print("="*70)
with open(model_filename, 'wb') as file:
    pickle.dump(model_svc_tuned, file)  # ✓ FIXED: Changed from model_knn_tuned
# Get file size
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)
print(f"✓ Model saved: {model_filename}")
print(f"File size: {file_size_mb:.4f} MB")
print(f"Best parameters: {svc_best_params}")  # ✓ FIXED: Changed from knn_best_params
print(f"Best CV score: {svc_best_cv_score:.4f}")  # ✓ FIXED: Changed from knn_best_cv_score
# Verify the saved model
print("\n--- VERIFYING SAVED MODEL ---")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
from sklearn.metrics import accuracy_score
y_pred_loaded = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"Loaded model test accuracy: {loaded_accuracy:.4f}")
print("✓ Model successfully saved and verified!")
print("\n" + "="*70)
print("PROCESS COMPLETE")
print("="*70)


HYPERPARAMETER TUNING: LINEAR SVC

GRID SEARCH: Linear SVC
Parameter grid: {'C': [100], 'penalty': ['l2'], 'loss': ['hinge'], 'max_iter': [1000]}
Cross-validation folds: 5
Scoring metric: F1-Score

Searching for optimal hyperparameters...
Fitting 5 folds for each of 1 candidates, totalling 5 fits

GRID SEARCH COMPLETE - Linear SVC
Search time: 45.46 seconds

Best Parameters:
  C: 100
  loss: hinge
  max_iter: 1000
  penalty: l2

Best CV F1-Score (Train): 85.65%


EVALUATING: Linear SVC (Tuned)

[1/5] Training model on training set...
[2/5] Cross-validation on training set (cv=5)...
[3/5] Evaluating on independent test set...
[4/5] Calculating deployment metrics...
[5/5] Results compiled.

              TRAINING SET PERFORMANCE (Cross-Validated)              
----------------------------------------------------------------------
  Accuracy:         85.39%
  Precision:        85.12%
  Recall:           85.78%
  F1-Score:         85.45%
  False Pos. Rate:  14.99%

            TEST SET PE

## 7. Results Comparison and Analysis

In [14]:
# Compile all results
all_results = {
    'Logistic Regression (Baseline)': results_lr_base,
    'Logistic Regression (Tuned)': results_lr_tuned,
    'KNN (Baseline)': results_knn_base,
    'KNN (Tuned)': results_knn_tuned,
    'Linear SVC (Baseline)': results_svc_base,
    'Linear SVC (Tuned)': results_svc_tuned
}

# Create comprehensive results DataFrame
results_data = []
for name, res in all_results.items():
    results_data.append({
        'Model': name,
        'Train_Accuracy': res['train_accuracy'],
        'Train_Precision': res['train_precision'],
        'Train_Recall': res['train_recall'],
        'Train_F1': res['train_f1'],
        'Train_FPR': res['train_fpr'],
        'Test_Accuracy': res['test_accuracy'],
        'Test_Precision': res['test_precision'],
        'Test_Recall': res['test_recall'],
        'Test_F1': res['test_f1'],
        'Test_FPR': res['test_fpr'],
        'Training_Time_sec': res['training_time_sec'],
        'Test_Time_sec': res['test_time_sec'],
        'Avg_Latency_ms': res['avg_latency_ms'],
        'Memory_MB': res['memory_mb'],
        'Model_Size_MB': res['model_size_mb'],
        'Throughput': res['throughput_samples_per_sec'],
        'Cost_Effectiveness': res['cost_effectiveness'],
        'Performance_per_MB': res['performance_per_mb']
    })

results_df = pd.DataFrame(results_data)

print("\n" + "="*100)
print("COMPREHENSIVE RESULTS SUMMARY")
print("="*100)
print("\n" + results_df.to_string(index=False))
print("\n" + "="*100)


COMPREHENSIVE RESULTS SUMMARY

                         Model  Train_Accuracy  Train_Precision  Train_Recall  Train_F1  Train_FPR  Test_Accuracy  Test_Precision  Test_Recall  Test_F1  Test_FPR  Training_Time_sec  Test_Time_sec  Avg_Latency_ms  Memory_MB  Model_Size_MB  Throughput  Cost_Effectiveness  Performance_per_MB
Logistic Regression (Baseline)           85.44            80.32         93.89     86.57      23.01          82.41           76.66        93.20    84.12     28.38              8.592          0.045          0.0006     182.86           0.00  1661368.35                9.79                0.46
   Logistic Regression (Tuned)           89.09            84.91         95.07     89.70      16.89          82.85           77.18        93.30    84.48     27.59             28.167          0.040          0.0005     251.45           0.00  1844208.80                3.00                0.34
                KNN (Baseline)           85.40            80.60         93.24     86.46      22.44

In [15]:
# Calculate improvements from tuning
improvements = []

for algo in ['Logistic Regression', 'KNN', 'Linear SVC']:
    base_f1 = results_df[results_df['Model'] == f"{algo} (Baseline)"]['Test_F1'].values[0]
    tuned_f1 = results_df[results_df['Model'] == f"{algo} (Tuned)"]['Test_F1'].values[0]
    improvement = tuned_f1 - base_f1
    
    improvements.append({
        'Algorithm': algo,
        'Baseline_Test_F1': base_f1,
        'Tuned_Test_F1': tuned_f1,
        'Improvement': improvement,
        'Improvement_Percent': round((improvement / base_f1) * 100, 2)
    })

improvements_df = pd.DataFrame(improvements)

print("\n" + "="*70)
print("HYPERPARAMETER TUNING IMPACT ON TEST SET")
print("="*70)
print("\n" + improvements_df.to_string(index=False))
print("\n" + "="*70)


HYPERPARAMETER TUNING IMPACT ON TEST SET

          Algorithm  Baseline_Test_F1  Tuned_Test_F1  Improvement  Improvement_Percent
Logistic Regression             84.12          84.48         0.36                 0.43
                KNN             86.91          88.35         1.44                 1.66
         Linear SVC             83.64          86.32         2.68                 3.20



In [22]:
import pickle
import numpy as np
import sys
import os
import tracemalloc
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
print("\n" + "="*70)
print("LOADING MODEL AND EVALUATING ON TEST SET")
print("="*70)
# Load the model from .pkl file
model_filename = '123SVC.pkl'  # Change this to your .pkl filename
print(f"\nLoading model from: {model_filename}")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
print("✓ Model loaded successfully")
# Get model memory usage
def get_size_mb(obj):
    size_bytes = sys.getsizeof(pickle.dumps(obj))
    return size_bytes / (1024 * 1024)
model_size_mb = get_size_mb(loaded_model)
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)
print(f"\n--- MODEL MEMORY USAGE ---")
print(f"Model memory size: {model_size_mb:.4f} MB")
print(f"File size on disk: {file_size_mb:.4f} MB")
# Display model information
print(f"\n--- MODEL INFORMATION ---")
print(f"Model type: {type(loaded_model).__name__}")
if hasattr(loaded_model, 'get_params'):
    params = loaded_model.get_params()
    print(f"Model parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
# Evaluate on TEST SET
print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)

# ✓ NEW: Measure prediction memory usage
print("\n--- PREDICTION MEMORY USAGE ---")
tracemalloc.start()

# Baseline memory
baseline_current, baseline_peak = tracemalloc.get_traced_memory()

# Make predictions
y_test_pred = loaded_model.predict(X_test)

# Memory after prediction
pred_current, pred_peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate memory used for prediction
pred_memory_mb = (pred_peak - baseline_peak) / (1024 * 1024)
pred_current_mb = (pred_current - baseline_current) / (1024 * 1024)

print(f"Peak memory for prediction: {pred_memory_mb:.4f} MB")
print(f"Current memory for prediction: {pred_current_mb:.4f} MB")
print(f"Test set size: {X_test.shape}")
print(f"Memory per sample: {(pred_memory_mb / len(X_test)) * 1024:.4f} KB")

# Calculate metrics
test_acc = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)
print(f"\n--- TEST METRICS ---")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
# ROC-AUC (if model supports probability predictions)
if hasattr(loaded_model, 'predict_proba'):
    print("\n--- PROBABILITY PREDICTION MEMORY ---")
    tracemalloc.start()
    baseline_current, baseline_peak = tracemalloc.get_traced_memory()
    
    y_test_proba = loaded_model.predict_proba(X_test)[:, 1]
    
    proba_current, proba_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    proba_memory_mb = (proba_peak - baseline_peak) / (1024 * 1024)
    print(f"Peak memory for predict_proba: {proba_memory_mb:.4f} MB")
    
    test_auc = roc_auc_score(y_test, y_test_proba)
    print(f"\nROC-AUC:   {test_auc:.4f}")
# Confusion Matrix
print("\n--- CONFUSION MATRIX ---")
cm = confusion_matrix(y_test, y_test_pred)
print(cm)
# Classification Report
print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_test_pred))
# ✓ NEW: Summary of all memory metrics
print("\n" + "="*70)
print("MEMORY SUMMARY")
print("="*70)
print(f"Model size in memory:     {model_size_mb:.4f} MB")
print(f"Model size on disk:       {file_size_mb:.4f} MB")
print(f"Prediction memory (peak): {pred_memory_mb:.4f} MB")
print(f"Memory per prediction:    {(pred_memory_mb / len(X_test)) * 1024:.4f} KB")
print(f"Total memory footprint:   {model_size_mb + pred_memory_mb:.4f} MB")
print("\n" + "="*70)
print("EVALUATION COMPLETE")
print("="*70)


LOADING MODEL AND EVALUATING ON TEST SET

Loading model from: 123SVC.pkl
✓ Model loaded successfully

--- MODEL MEMORY USAGE ---
Model memory size: 0.0046 MB
File size on disk: 0.0046 MB

--- MODEL INFORMATION ---
Model type: LinearSVC
Model parameters:
  C: 100
  class_weight: None
  dual: auto
  fit_intercept: True
  intercept_scaling: 1
  loss: hinge
  max_iter: 1000
  multi_class: ovr
  penalty: l2
  random_state: 42
  tol: 0.0001
  verbose: 0

TEST SET PERFORMANCE

--- PREDICTION MEMORY USAGE ---
Peak memory for prediction: 110.6421 MB
Current memory for prediction: 0.5660 MB
Test set size: (74000, 194)
Memory per sample: 1.5310 KB

--- TEST METRICS ---
Accuracy:  0.8711
Precision: 0.8761
Recall:    0.8711
F1-Score:  0.8707

--- CONFUSION MATRIX ---
[[34363  2637]
 [ 6901 30099]]

--- CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.83      0.93      0.88     37000
           1       0.92      0.81      0.86     37000

    accu

In [23]:
import pickle
import numpy as np
import sys
import os
import tracemalloc
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
print("\n" + "="*70)
print("LOADING MODEL AND EVALUATING ON TEST SET")
print("="*70)
# Load the model from .pkl file
model_filename = '123Logistic.pkl'  # Change this to your .pkl filename
print(f"\nLoading model from: {model_filename}")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
print("✓ Model loaded successfully")
# Get model memory usage
def get_size_mb(obj):
    size_bytes = sys.getsizeof(pickle.dumps(obj))
    return size_bytes / (1024 * 1024)
model_size_mb = get_size_mb(loaded_model)
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)
print(f"\n--- MODEL MEMORY USAGE ---")
print(f"Model memory size: {model_size_mb:.4f} MB")
print(f"File size on disk: {file_size_mb:.4f} MB")
# Display model information
print(f"\n--- MODEL INFORMATION ---")
print(f"Model type: {type(loaded_model).__name__}")
if hasattr(loaded_model, 'get_params'):
    params = loaded_model.get_params()
    print(f"Model parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
# Evaluate on TEST SET
print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)

# ✓ NEW: Measure prediction memory usage
print("\n--- PREDICTION MEMORY USAGE ---")
tracemalloc.start()

# Baseline memory
baseline_current, baseline_peak = tracemalloc.get_traced_memory()

# Make predictions
y_test_pred = loaded_model.predict(X_test)

# Memory after prediction
pred_current, pred_peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate memory used for prediction
pred_memory_mb = (pred_peak - baseline_peak) / (1024 * 1024)
pred_current_mb = (pred_current - baseline_current) / (1024 * 1024)

print(f"Peak memory for prediction: {pred_memory_mb:.4f} MB")
print(f"Current memory for prediction: {pred_current_mb:.4f} MB")
print(f"Test set size: {X_test.shape}")
print(f"Memory per sample: {(pred_memory_mb / len(X_test)) * 1024:.4f} KB")

# Calculate metrics
test_acc = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)
print(f"\n--- TEST METRICS ---")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
# ROC-AUC (if model supports probability predictions)
if hasattr(loaded_model, 'predict_proba'):
    print("\n--- PROBABILITY PREDICTION MEMORY ---")
    tracemalloc.start()
    baseline_current, baseline_peak = tracemalloc.get_traced_memory()
    
    y_test_proba = loaded_model.predict_proba(X_test)[:, 1]
    
    proba_current, proba_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    proba_memory_mb = (proba_peak - baseline_peak) / (1024 * 1024)
    print(f"Peak memory for predict_proba: {proba_memory_mb:.4f} MB")
    
    test_auc = roc_auc_score(y_test, y_test_proba)
    print(f"\nROC-AUC:   {test_auc:.4f}")
# Confusion Matrix
print("\n--- CONFUSION MATRIX ---")
cm = confusion_matrix(y_test, y_test_pred)
print(cm)
# Classification Report
print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_test_pred))
# ✓ NEW: Summary of all memory metrics
print("\n" + "="*70)
print("MEMORY SUMMARY")
print("="*70)
print(f"Model size in memory:     {model_size_mb:.4f} MB")
print(f"Model size on disk:       {file_size_mb:.4f} MB")
print(f"Prediction memory (peak): {pred_memory_mb:.4f} MB")
print(f"Memory per prediction:    {(pred_memory_mb / len(X_test)) * 1024:.4f} KB")
print(f"Total memory footprint:   {model_size_mb + pred_memory_mb:.4f} MB")
print("\n" + "="*70)
print("EVALUATION COMPLETE")
print("="*70)


LOADING MODEL AND EVALUATING ON TEST SET

Loading model from: 123Logistic.pkl
✓ Model loaded successfully

--- MODEL MEMORY USAGE ---
Model memory size: 0.0047 MB
File size on disk: 0.0047 MB

--- MODEL INFORMATION ---
Model type: LogisticRegression
Model parameters:
  C: 100
  class_weight: None
  dual: False
  fit_intercept: True
  intercept_scaling: 1
  l1_ratio: None
  max_iter: 1000
  multi_class: deprecated
  n_jobs: -1
  penalty: l2
  random_state: 42
  solver: liblinear
  tol: 0.0001
  verbose: 0
  warm_start: False

TEST SET PERFORMANCE

--- PREDICTION MEMORY USAGE ---
Peak memory for prediction: 110.6421 MB
Current memory for prediction: 0.5661 MB
Test set size: (74000, 194)
Memory per sample: 1.5310 KB

--- TEST METRICS ---
Accuracy:  0.8285
Precision: 0.8435
Recall:    0.8285
F1-Score:  0.8266

--- PROBABILITY PREDICTION MEMORY ---
Peak memory for predict_proba: 110.6591 MB

ROC-AUC:   0.9573

--- CONFUSION MATRIX ---
[[26790 10210]
 [ 2478 34522]]

--- CLASSIFICATION REPO

In [24]:
import pickle
import numpy as np
import sys
import os
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

print("\n" + "="*70)
print("LOADING MODEL AND EVALUATING ON TEST SET")
print("="*70)

# Load the model from .pkl file
model_filename = '123knn_tuned_model.pkl'  # Change this to your .pkl filename

print(f"\nLoading model from: {model_filename}")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

print("✓ Model loaded successfully")

# Get model memory usage
def get_size_mb(obj):
    size_bytes = sys.getsizeof(pickle.dumps(obj))
    return size_bytes / (1024 * 1024)

model_size_mb = get_size_mb(loaded_model)
file_size_mb = os.path.getsize(model_filename) / (1024 * 1024)

print(f"\n--- MEMORY USAGE ---")
print(f"Model memory size: {model_size_mb:.4f} MB")
print(f"File size on disk: {file_size_mb:.4f} MB")

# Display model information
print(f"\n--- MODEL INFORMATION ---")
print(f"Model type: {type(loaded_model).__name__}")
if hasattr(loaded_model, 'get_params'):
    params = loaded_model.get_params()
    print(f"Model parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")

# Evaluate on TEST SET
print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)

y_test_pred = loaded_model.predict(X_test)

# Calculate metrics
test_acc = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)

print(f"\nAccuracy:  {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")

# ROC-AUC (if model supports probability predictions)
if hasattr(loaded_model, 'predict_proba'):
    y_test_proba = loaded_model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_proba)
    print(f"ROC-AUC:   {test_auc:.4f}")

# Confusion Matrix
print("\n--- CONFUSION MATRIX ---")
cm = confusion_matrix(y_test, y_test_pred)
print(cm)

# Classification Report
print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_test_pred))

print("\n" + "="*70)
print("EVALUATION COMPLETE")
print("="*70)


LOADING MODEL AND EVALUATING ON TEST SET

Loading model from: 123knn_tuned_model.pkl
✓ Model loaded successfully

--- MEMORY USAGE ---
Model memory size: 166.6292 MB
File size on disk: 166.6291 MB

--- MODEL INFORMATION ---
Model type: KNeighborsClassifier
Model parameters:
  algorithm: auto
  leaf_size: 30
  metric: manhattan
  metric_params: None
  n_jobs: -1
  n_neighbors: 11
  p: 2
  weights: distance

TEST SET PERFORMANCE

Accuracy:  0.8768
Precision: 0.8819
Recall:    0.8768
F1-Score:  0.8764
ROC-AUC:   0.9656

--- CONFUSION MATRIX ---
[[30318  6682]
 [ 2432 34568]]

--- CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.93      0.82      0.87     37000
           1       0.84      0.93      0.88     37000

    accuracy                           0.88     74000
   macro avg       0.88      0.88      0.88     74000
weighted avg       0.88      0.88      0.88     74000


EVALUATION COMPLETE
